In [1]:
import re
import codecs
import itertools
import os
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from string import punctuation


In [2]:
from transformers import BertTokenizer, BertModel
import torch
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch.nn.utils.rnn as rnn_utils
import os
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

from nltk.stem import PorterStemmer
porter_stemmer = PorterStemmer()
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import numpy as np
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
from __future__ import division
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
import sys
import os
from os.path import isfile, join
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
def read_input_file(input_file , mode="r" , encoding='utf-8'):
    
    text = ""

    if os.path.exists(input_file) :
        with codecs.open(input_file , mode , encoding) as f :
            text = f.read()
        
        f.close()
        return text
    
    return None

In [6]:
def read_gold_file(gold_file , mode="r" , encoding='utf-8'):

    text = ""

    if os.path.exists(gold_file) :
        with codecs.open(gold_file , mode , encoding) as f :
            text = f.readlines()
        
        f.close()
        return text
    
    return None

In [7]:
def tokenize(text):
    
    return text.split()

In [8]:
def get_dataset(abstract_file_path , keyphrases_file_path):
    
    abstract_dataset = []
    keyphrases_dataset = []
    
    files = [f for f in os.listdir(abstract_file_path) if isfile(join(abstract_file_path, f))]
    i =0
    
    
    for file in files:
            
        abstract_file = abstract_file_path + "/" + file 
        keyphrases_file = keyphrases_file_path + "/" + file 
        
        abtract = read_input_file(abstract_file)
        keyphrases = read_gold_file(keyphrases_file)
        
        new_keyphrases = []
        
        for key in keyphrases:
            splitted_key = key.split("!!!")
            new_keyphrases.append(splitted_key[0])
        
#         if  i ==0:
#             i = 2
#             print(file)
#             print(type(keyphrases))
#             print(keyphrases)
#             print("==================================")
#             print(new_keyphrases)
#             print(type(new_keyphrases))
        
        abstract_dataset.append(abtract)
        keyphrases_dataset.append(new_keyphrases)
        
        
    return abstract_dataset , keyphrases_dataset

In [9]:
train_abstract_folder_path = '/kaggle/input/keyphrases-exctraction-seam-eval2017/preprocessed_dataset/train/paragraphs'
train_keyphrases_folder_path = '/kaggle/input/keyphrases-exctraction-seam-eval2017/preprocessed_dataset/train/keyphrases'
val_abstract_folder_path = '/kaggle/input/keyphrases-exctraction-seam-eval2017/preprocessed_dataset/val/paragraphs'
val_keyphrases_folder_path = '/kaggle/input/keyphrases-exctraction-seam-eval2017/preprocessed_dataset/val/keyphrases'
test_abstract_folder_path = '/kaggle/input/keyphrases-exctraction-seam-eval2017/preprocessed_dataset/test/paragraphs'
test_keyphrases_folder_path = '/kaggle/input/keyphrases-exctraction-seam-eval2017/preprocessed_dataset/test/keyphrases'


In [10]:
train_abstract , train_keyphrases = get_dataset(train_abstract_folder_path , train_keyphrases_folder_path)
val_abstract , val_keyphrases = get_dataset(val_abstract_folder_path , val_keyphrases_folder_path)
test_abstract , test_keyphrases = get_dataset(test_abstract_folder_path , test_keyphrases_folder_path)

In [11]:
print("The size of Train: " , len(train_abstract))
print("The size of Validation: " , len(val_abstract))
print("The size of Test: " , len(test_abstract))

The size of Train:  350
The size of Validation:  50
The size of Test:  100


In [12]:
# val_abstract = val_abstract[:4]
# val_keyphrases = val_keyphrases[:4]

In [13]:
# val_keyphrases

In [14]:
import torch
from transformers import BertTokenizer, BertForTokenClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [15]:
import torch
from transformers import DebertaTokenizer, DebertaForTokenClassification, AdamW, AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v2-xlarge")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [17]:
def masking_and_labeling(abstract_dataset , keyphrases_dataset):
        
    tokenized_texts = []
    formatted_labels_list = []

    for abstract, keyphrase_list in zip(abstract_dataset, keyphrases_dataset):
        tokenized_abstract = tokenizer.tokenize(abstract)
        tokenized_texts.append(tokenized_abstract)

        formatted_label = [0] * len(tokenized_abstract)  # Initialize labels with zeros

        for keyphrase in keyphrase_list:
            keyphrase_tokens = tokenizer.tokenize(keyphrase)
            for i in range(len(tokenized_abstract) - len(keyphrase_tokens) + 1):
                if tokenized_abstract[i:i + len(keyphrase_tokens)] == keyphrase_tokens:
                    for j in range(i, i + len(keyphrase_tokens)):
                        formatted_label[j] = 1  # Set labels of keyphrase tokens to 1
    #                 break
        formatted_labels_list.append(formatted_label)
    
    return tokenized_texts , formatted_labels_list

In [18]:
train_tokenized_texts , train_labels = masking_and_labeling(train_abstract , train_keyphrases)
val_tokenized_texts , val_labels = masking_and_labeling(val_abstract , val_keyphrases)
test_tokenized_texts , test_labels = masking_and_labeling(test_abstract , test_keyphrases)

In [19]:
len(train_tokenized_texts)

350

In [20]:
def dataset(tokenized_texts , formatted_labels_list):
    max_length = max(len(txt) for txt in tokenized_texts)
    input_ids = torch.tensor([tokenizer.convert_tokens_to_ids(txt + ['[PAD]'] * (max_length - len(txt))) for txt in tokenized_texts])

    labels = torch.tensor([lbl + [0] * (max_length - len(lbl)) for lbl in formatted_labels_list])
    
    masks = torch.where(input_ids != 0, torch.tensor(1), torch.tensor(0))
    
    return input_ids , labels, masks

In [21]:
train_inputs , train_labels , train_mask = dataset(train_tokenized_texts , train_labels)
val_inputs , val_labels , val_mask = dataset(val_tokenized_texts , val_labels)
test_inputs  , test_labels , test_mask = dataset(test_tokenized_texts , test_labels)

In [22]:
id_to_token = {token_id: token for token, token_id in tokenizer.vocab.items()}
len(id_to_token)

30522

In [23]:
train_data = TensorDataset(train_inputs, train_mask, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=16)


In [24]:
validation_data = TensorDataset(val_inputs, val_mask, val_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=16)

In [25]:
test_data = TensorDataset(test_inputs, test_mask, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=16)

In [26]:
from transformers import AutoTokenizer, DebertaV2Model

In [27]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,  # binary classification (keyphrase or not)
    output_attentions=False,
    output_hidden_states=False
)


# model = DebertaV2Model.from_pretrained("microsoft/deberta-v2-xlarge",
#    num_labels=2,  # binary classification (keyphrase or not)
#     output_attentions=False,
#     output_hidden_states=False)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [29]:
from transformers.modeling_outputs import TokenClassifierOutput

In [30]:
# decoded_sentence = tokenizer.convert_tokens_to_string(pred_phrase) 

In [31]:
def return_keyphrase(sentence , logits_list , original_sequence):
    
    predited_sequence = []
    batch_size = len(logits_list)
    
    for b in range(batch_size):
        
        batch_res = []
        logits_element = logits_list[b]
        
        for logit in logits_element:
        
            zero_class_logit = logit[0]
            one_class_logit = logit[1]

            if zero_class_logit > one_class_logit:
                batch_res.append(0)
            else:
                batch_res.append(1)
                
        predited_sequence.append(batch_res)
    
    predicted_keyphrases = []
    
    for b in range(batch_size):
        
        pred_seq = predited_sequence[b]
        sent = sentence[b]        
        sent_len = len(sent)
        
        pred_phrase = []        
        list_of_pred_phrase = []
        
        for index in range(sent_len):
            
            logit = pred_seq[index]
            word_index = sent[index]
            word = id_to_token[word_index]
            
            if logit == 0:
                
                if len(pred_phrase) < 1:
                    continue
                
                decoded_sentence = tokenizer.convert_tokens_to_string(pred_phrase)       
                list_of_pred_phrase.append(decoded_sentence)
                
                pred_phrase = []
            
            else:
                pred_phrase.append(word)
            
        if len(pred_phrase) > 0:
            decoded_sentence = tokenizer.convert_tokens_to_string(pred_phrase)       
            list_of_pred_phrase.append(decoded_sentence)
            
        predicted_keyphrases.append(list_of_pred_phrase)  
        
        
    original_keyphrases = []
    
    for b in range(batch_size):
        
        original_seq = original_sequence[b]
        sent = sentence[b]        
        sent_len = len(sent)
        
        original_phrase = []        
        list_of_original_phrase = []
        
        for index in range(sent_len):
            
            logit = original_seq[index]
            word_index = sent[index]
            word = id_to_token[word_index]
            
            if logit == 0:
                
                if len(original_phrase) < 1:
                    continue
                
                decoded_sentence = tokenizer.convert_tokens_to_string(original_phrase)       
                list_of_original_phrase.append(decoded_sentence)
                
                original_phrase = []
            
            else:
                original_phrase.append(word)
            
        if len(original_phrase) > 0:
            decoded_sentence = tokenizer.convert_tokens_to_string(original_phrase)       
            list_of_original_phrase.append(decoded_sentence)
            
        original_keyphrases.append(list_of_original_phrase)
    
    return predicted_keyphrases , original_keyphrases
    
# # Get word from token ID
# word = id_to_token[1996]
# temp = []

# for t in input_ids[0]:
#     temp.append(id_to_token[t.item()])

In [32]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [33]:
epochs = 50
best_f1_score  = 0.0
best_recall = 0.0
best_precision = 0.0

In [34]:


for epoch in range(epochs):
    
    
    model.train()
    total_loss = 0
    
    for batch in tqdm(train_dataloader, desc="Epoch {}/{}".format(epoch + 1 , epochs )):
        
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        
        optimizer.zero_grad()
        outputs = model(**inputs)
        
#         logits_list = outputs.logits.tolist()

        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print("Average training loss: {:.4f}".format(avg_train_loss))

    model.eval()
    eval_loss = 0
    nb_eval_steps = 0
    
    predicted_keyphrases_list = []
    original_keyphrases_list = []

    for batch in tqdm(validation_dataloader, desc="Validation"):
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
            outputs = model(**inputs)
            tmp_eval_loss = outputs.loss
            eval_loss += tmp_eval_loss.mean().item()
            
            predicted_labels = torch.argmax(outputs.logits, axis=-1)
            predicted_keyphrases , original_keyphrases = return_keyphrase(batch[0].tolist() , outputs.logits.tolist() , batch[2].tolist())
            
            for keyphrase in predicted_keyphrases:
                predicted_keyphrases_list.append(keyphrase)
            
            for orig_keyphrase in original_keyphrases:
                original_keyphrases_list.append(orig_keyphrase)
            
        nb_eval_steps += 1

    avg_val_loss = eval_loss / len(validation_dataloader)
    print("Average validation loss: {:.4f}".format(avg_val_loss))
    
    total_prediction = 0
    total_original = 0
    total_correct_prediction = 0
    total_docs = 0
    
    preicision = 0.0
    recall = 0.0
    f1_score = 0.0
    
    for index in range(len(original_keyphrases_list)):
        
        predicted = predicted_keyphrases_list[index]
        original = original_keyphrases_list[index]
        
        set_predicted = set(predicted)
        set_original = set(original)
        
        correct_prediction = set_original.intersection(set_predicted)
        
        count_prediction = len(set_predicted)
        count_original = len(set_original)
        count_correct_prediction = len(correct_prediction)
        
        docs_precision = count_correct_prediction / count_prediction  if count_prediction > 0 else 0.0
        docs_recall = count_correct_prediction / count_original  if count_original > 0  else 0.0
        docs_f1_score = 2 * docs_precision * docs_recall / (docs_precision + docs_recall) if (docs_precision + docs_recall) > 0.0 else 0.0
        
        preicision += docs_precision
        recall += docs_recall
        f1_score += docs_f1_score

        total_docs += 1
    
    final_precision = preicision/total_docs
    final_recall = recall/total_docs
    final_f1_score = f1_score/total_docs

     
    if final_f1_score > best_f1_score:
        best_precision = final_precision
        best_recall = final_recall
        best_f1_score = final_f1_score

        torch.save(model.state_dict(), 'best_model.pth')
        
    print("Validation Precision : {:.4f}".format(final_precision))
    print("Validation recall : {:.4f}".format(final_recall))    
    print("Validation f1_score : {:.4f}".format(final_f1_score))    
    print("================================================================================")

Epoch 1/50: 100%|██████████| 22/22 [00:16<00:00,  1.30it/s]


Average training loss: 0.3552


Validation: 100%|██████████| 4/4 [00:00<00:00,  4.44it/s]


Average validation loss: 0.1762
Validation Precision : 0.3605
Validation recall : 0.5807
Validation f1_score : 0.4374


Epoch 2/50: 100%|██████████| 22/22 [00:16<00:00,  1.35it/s]


...

Average validation loss: 0.3712
Validation Precision : 0.5020
Validation recall : 0.6068
Validation f1_score : 0.5414


Epoch 50/50: 100%|██████████| 22/22 [00:16<00:00,  1.34it/s]


Average training loss: 0.0018


Validation: 100%|██████████| 4/4 [00:00<00:00,  4.47it/s]

Average validation loss: 0.4100
Validation Precision : 0.5103
Validation recall : 0.6114
Validation f1_score : 0.5469


In [35]:
print("Best Validation Precision : {:.4f}".format(best_precision))
print("Best Validation recall : {:.4f}".format(best_recall))    
print("Best Validation f1_score : {:.4f}".format(best_f1_score))  

Best Validation Precision : 0.5348
Best Validation recall : 0.6076
Best Validation f1_score : 0.5614


In [36]:
best_model = BertForTokenClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,  # binary classification (keyphrase or not)
    output_attentions=False,
    output_hidden_states=False
).to(device)
best_model.load_state_dict(torch.load('best_model.pth'))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [37]:
best_model.eval()
test_loss = 0
nb_test_steps = 0

predicted_keyphrases_list = []
original_keyphrases_list = []

for batch in tqdm(test_dataloader, desc="Test"):
    
    batch = tuple(t.to(device) for t in batch)
    
    with torch.no_grad():
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        outputs = best_model(**inputs)
        tmp_test_loss = outputs.loss
        test_loss += tmp_test_loss.mean().item()

        predicted_labels = torch.argmax(outputs.logits, axis=-1)
        predicted_keyphrases , original_keyphrases = return_keyphrase(batch[0].tolist() , outputs.logits.tolist() , batch[2].tolist())

        for keyphrase in predicted_keyphrases:
            predicted_keyphrases_list.append(keyphrase)

        for orig_keyphrase in original_keyphrases:
            original_keyphrases_list.append(orig_keyphrase)

    nb_test_steps += 1

avg_test_loss = test_loss / len(test_dataloader)
print("Average Test loss: {:.4f}".format(avg_val_loss))

total_prediction = 0
total_original = 0
total_correct_prediction = 0

for index in range(len(original_keyphrases_list)):

    predicted = predicted_keyphrases_list[index]
    original = original_keyphrases_list[index]

    set_predicted = set(predicted)
    set_original = set(original)

    correct_prediction = set_original.intersection(set_predicted)

    count_prediction = len(set_predicted)
    count_original = len(set_original)
    count_correct_prediction = len(correct_prediction)

    total_prediction += count_prediction
    total_original += count_original
    total_correct_prediction += count_correct_prediction


precision , recall, f1_socre = 0.0 , 0.0 , 0.0

precision =  total_correct_prediction / total_prediction  if total_prediction > 0 else 0.0
recall = total_correct_prediction / total_original  if total_original > 0  else 0.0
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0.0 else 0.0

print("Test Precision : {:.4f}".format(precision))
print("Test recall : {:.4f}".format(recall))    
print("Test f1_score : {:.4f}".format(f1_score))    
print("================================================================================")

Test: 100%|██████████| 7/7 [00:01<00:00,  4.58it/s]

Average Test loss: 0.4100
Test Precision : 0.4912
Test recall : 0.5924
Test f1_score : 0.5371


In [38]:
def find_keyphrases_at_interfernce(abstract , model_file_path):
    
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    
    tokenized_abstract = tokenizer.tokenize(abstract)
    max_length = max(len(txt) for txt in tokenized_abstract)
    
    input_ids = torch.tensor([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_abstract])

    
    masks = torch.where(input_ids != 0, torch.tensor(1), torch.tensor(0))
    id_to_token = {token_id: token for token, token_id in tokenizer.vocab.items()}

    
    interfernce_data = TensorDataset(input_ids, masks)
    interfernce_sampler = RandomSampler(interfernce_data)
    interfernce_dataloader = DataLoader(interfernce_data, sampler=test_sampler, batch_size=1)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    
    model = BertForTokenClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=2, 
        output_attentions=False,
        output_hidden_states=False
    ).to(device)
    
    model.load_state_dict(torch.load(model_file_path))
    model.eval()
    
    
    for batch in tqdm(interfernce_dataloader, desc="Interfernce"):
    
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {"input_ids": batch[0], "attention_mask": batch[1]}
            outputs = model(**inputs)
    
            predicted_keyphrases = return_keyphrase(batch[0].tolist() , outputs.logits.tolist())
            
            predited_sequence = []
            batch_size = len(logits_list)

            for b in range(batch_size):

                batch_res = []
                logits_element = logits_list[b]

                for logit in logits_element:

                    zero_class_logit = logit[0]
                    one_class_logit = logit[1]

                    if zero_class_logit > one_class_logit:
                        batch_res.append(0)
                    else:
                        batch_res.append(1)

                predited_sequence.append(batch_res)

            predicted_keyphrases = []

            for b in range(batch_size):

                pred_seq = predited_sequence[b]
                sent = sentence[b]        
                sent_len = len(sent)

                pred_phrase = []        
                list_of_pred_phrase = []

                for index in range(sent_len):

                    logit = pred_seq[index]
                    word_index = sent[index]
                    word = id_to_token[word_index]

                    if logit == 0:

                        if len(pred_phrase) < 1:
                            continue

                        decoded_sentence = tokenizer.convert_tokens_to_string(pred_phrase)       
                        list_of_pred_phrase.append(decoded_sentence)

                        pred_phrase = []

                    else:
                        pred_phrase.append(word)

                if len(pred_phrase) > 0:
                    decoded_sentence = tokenizer.convert_tokens_to_string(pred_phrase)       
                    list_of_pred_phrase.append(decoded_sentence)

                predicted_keyphrases.append(list_of_pred_phrase)  
    
    return predicted_keyphrases

In [39]:
abstract = "Fig. 9 displays the growth of two of the main corrosion products that develop or form on the surface of Cu40Zn with time, hydrozincite (Fig. 9a) and Cu2O (Fig. 9b). It should be remembered that both phases were present already from start of the exposure. The data is presented in absorbance units and allows comparisons to be made of the amounts of each species between the two Cu40Zn surfaces investigated, DP and HZ7. The tendency is very clear that the formation rates of both hydrozincite and cuprite are quite suppressed for Cu40Zn with preformed hydrozincite (HZ7) compared to the diamond polished surface (DP). In summary, without being able to consider the formation of simonkolleite, it can be concluded that an increased surface coverage of hydrozincite reduces the initial spreading ability of the NaCl-containing droplets and thereby lowers the overall formation rate of hydrozincite and cuprite."

In [40]:
def predict_keyphrases(model, tokenizer, abstract):
    model.eval()
    
    tokenized_abstract = tokenizer.tokenize(abstract)

    input_ids = tokenizer.convert_tokens_to_ids(tokenized_abstract)
    
    # Add padding to match the maximum sequence length
    max_length = model.config.max_position_embeddings
    input_ids += [tokenizer.pad_token_id] * (max_length - len(input_ids))
    
    # Convert to tensor
    input_ids_tensor = torch.tensor(input_ids).unsqueeze(0).to(device)  # Add batch dimension
    
    # Perform forward pass
    with torch.no_grad():
        outputs = model(input_ids_tensor)
    
    # Get predicted labels
    predicted_labels = torch.argmax(outputs.logits, axis=-1).squeeze(0).tolist()
    
    # Decode predicted keyphrases
    predicted_keyphrases = []
    current_keyphrase = []
    for i, token_id in enumerate(input_ids):
        token = tokenizer.convert_ids_to_tokens(token_id)
        if token == '[PAD]':
            break
        if predicted_labels[i] == 1:  # Token belongs to keyphrase
            current_keyphrase.append(token)
        else:
            if current_keyphrase:
                predicted_keyphrases.append(tokenizer.convert_tokens_to_string(current_keyphrase))
                current_keyphrase = []
    
    return predicted_keyphrases

# Example usage
# abstract = "This is a sample abstract about natural language processing."
predicted_keyphrases = list(set(predict_keyphrases(best_model, tokenizer, abstract)))
print("Predicted Keyphrases:", predicted_keyphrases)


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Predicted Keyphrases: ['simonkolleite', 'cu2o', 'nacl - containing droplets', 'pre', 'hz7', 'hydrozincite', 'dp', 'cu40zn', 'cu40zn surfaces', 'absorbance units', 'diamond polished surface', 'cuprite', 'dp and hz7', 'corrosion products']


In [41]:
len(predicted_keyphrases)

14